## Baseline:

This is a MLE baseline for the gender reinflection task. The baseline models $argmax$ $p(trg_w | src_w, trg_g)$, where $trg_w$ is the target word, $src_w$ is the source word and $trg_g$ is the target gender. This baseline makes sense because the source sentence and the target sentence always have the same length and perfect alignment. So to implement this model, we will simply implement the following: $$p(trg_w | src_w, trg_g) = \dfrac{count(trg_w, src_w, trg_g)}{count(src_w, trg_g)}$$

In [54]:
from collections import defaultdict
import copy
import json
import os
import operator
import sacrebleu
import os

In [55]:
class InputExample:
    """Simple object to encapsulate each data example"""
    def __init__(self, src, trg, 
                 src_g, trg_g):    
        self.src = src
        self.trg = trg
        self.src_g = src_g
        self.trg_g = trg_g
    
    def __repr__(self):
        return str(self.to_json_str())
    
    def to_json_str(self):
        return json.dumps(self.to_dict(), indent=2, ensure_ascii=False)
    
    def to_dict(self):
        output = copy.deepcopy(self.__dict__)
        return output

In [56]:
class RawDataset:
    """Encapsulates the raw examples in InputExample objects"""
    def __init__(self, data_dir):
        self.train_examples = self.get_train_examples(data_dir)
        self.dev_examples = self.get_dev_examples(data_dir)
        self.test_examples = self.get_dev_examples(data_dir)
        
    def create_examples(self, src_path, trg_path):
        
        src_txt = self.get_txt_examples(src_path)
        src_gender_labels = self.get_labels(src_path + '.label')
        trg_txt = self.get_txt_examples(trg_path)
        trg_gender_labels = self.get_labels(trg_path + '.label')
        
        examples = []
        
        for i in range(len(src_txt)):
            src = src_txt[i].strip()
            trg = trg_txt[i].strip()
            src_g = src_gender_labels[i].strip()
            trg_g = trg_gender_labels[i].strip()
            input_example = InputExample(src, trg, src_g, trg_g)
            examples.append(input_example)
        
        return examples
    
    def get_labels(self, data_dir):
        with open(data_dir) as f:
            return f.readlines()
        
    def get_txt_examples(self, data_dir):
        with open(data_dir, encoding='utf8') as f:
            return f.readlines()
    
    def get_train_examples(self, data_dir):
        """Reads the train examples of the dataset"""
        return self.create_examples(os.path.join(data_dir, 'D-set-train.arin'), 
                                    os.path.join(data_dir, 'D-set-train.ar.M'))
    
    def get_dev_examples(self, data_dir):
        """Reads the dev examples of the dataset"""
        return self.create_examples(os.path.join(data_dir, 'D-set-dev.arin'), 
                                    os.path.join(data_dir, 'D-set-dev.ar.M'))
    
    def get_test_examples(self, data_dir):
        """Reads the test examples of the dataset"""
        return self.create_examples(os.path.join(data_dir, 'D-set-test.arin'), 
                                    os.path.join(data_dir, 'D-set-test.ar.M'))

In [57]:
dataset = RawDataset('/home/ba63/gender-bias/christine_2019/Arabic-parallel-gender-corpus/')


In [58]:
train_examples = dataset.train_examples
len(train_examples)

8566

In [59]:
train_examples[8557]

{
  "src": "عمل تقتضيه العلاقات العامة و كنت قلقة من ردة فعلها حول ميمي و التي عادة ما تتكبر على أي امرأة أخرى",
  "trg": "عمل تقتضيه العلاقات العامة و كنت قلقا من ردة فعلها حول ميمي و التي عادة ما تتكبر على أي امرأة أخرى",
  "src_g": "F",
  "trg_g": "M"
}

In [60]:
def build_ngrams(sentence, ngrams=2):
    """
    Args:
     - sentence (str): a list of words
     - ngrams (int): 2 for bigrams, 3 for trigrams, etc..
    Returns:
     - ngrams list of the sentece
    """
    return [sentence[i - (ngrams - 1): i + 1] for i in range(ngrams - 1, len(sentence))]

In [61]:
class MLE:
    """MLE to model P(t_w | s_w, t_g)"""
    
    def __init__(self, examples):
        self.model = self.build_model(examples)
        
    def build_model(self, examples):
        """
        Args:
            - examples (list): list of InputExample objects
        Returns:
            - model (dict): 
        """
        
        model = defaultdict(lambda: defaultdict(lambda: 0))
        self.context_counts = dict()
        for i, ex in enumerate(examples):

            src = ex.src
            trg = ex.trg
            src_g = ex.src_g
            trg_g = ex.trg_g
            src = src.split(' ')
            trg = trg.split(' ')
            
            for i, trg_w in enumerate(trg):
                # counts of (t_w, s_w, t_g)
                model[(src[i], trg_g)][trg_w] += 1
                # counts of (s_w, t_g)
                self.context_counts[(src[i], trg_g)] = 1 +  self.context_counts.get((src[i], trg_g), 0)
            
        # turning the counts into probs
        for sw, trg_g in model:
            for trg_w in model[(sw, trg_g)]:
                model[(sw, trg_g)][trg_w] = float(model[(sw, trg_g)][trg_w]) / self.context_counts[(sw, trg_g)] 
        
        return model
    
    def __getitem__(self, sw_tg):
        if sw_tg in self.model:
            return dict(self.model[sw_tg])
        else:
            return {'<unk>': 0.0}
    
    def __len__(self):
        return len(self.model)
    

In [62]:
mle = MLE(train_examples)
print(len(mle))

15059


In [63]:
def generate_sentence(mle, src, trg_g):
    """Generates a sentence based on the mle model. 
    At each time step, the model will pick the word with maximum prob"""
    src = src.split(' ')
    target = []
    for sw in src:
        candidates = mle[(sw, trg_g)]
#         print(candidates)
        argmax = max(candidates.items(), key=operator.itemgetter(1))[0]
        target.append(argmax)

    return ' '.join(target)

#### Interesting Examples from the D-set-train.arin (input) dataset: <br/>
`generate_sentence(mle, 'أصبحت نظيفة .', 'M')`<br/>
`gold: أصبحت نظيفا .`

`generate_sentence(mle, 'عمل تقتضيه العلاقات العامة و كنت قلقة من ردة فعلها حول ميمي و التي عادة ما تتكبر على أي امرأة أخرى', 'M')`<br/>
`gold: عمل تقتضيه العلاقات العامة و كنت قلقا من ردة فعلها حول ميمي و التي عادة ما تتكبر على أي امرأة أخرى`

In [43]:
generate_sentence(mle, 'أصبحت نظيفة .', 'M')

{'أصبحت': 1.0}
{'نظيف': 0.25, 'نظيفة': 0.5, 'نظيفا': 0.25}
{'.': 1.0}


'أصبحت نظيفة .'

In [44]:
generate_sentence(mle, 'عمل تقتضيه العلاقات العامة و كنت قلقة من ردة فعلها حول ميمي و التي عادة ما تتكبر على أي امرأة أخرى', 'M')

{'عمل': 1.0}
{'تقتضيه': 1.0}
{'العلاقات': 1.0}
{'العامة': 1.0}
{'و': 1.0}
{'كنت': 1.0}
{'قلق': 0.75, 'قلقا': 0.25}
{'من': 1.0}
{'ردة': 1.0}
{'فعلها': 1.0}
{'حول': 1.0}
{'ميمي': 1.0}
{'و': 1.0}
{'التي': 0.75, 'الذي': 0.25}
{'عادة': 1.0}
{'ما': 1.0}
{'تتكبر': 1.0}
{'على': 1.0}
{'أي': 1.0}
{'رجل': 0.5555555555555556, 'رجلا': 0.2777777777777778, 'امرأة': 0.16666666666666666}
{'أخرى': 1.0}


'عمل تقتضيه العلاقات العامة و كنت قلق من ردة فعلها حول ميمي و التي عادة ما تتكبر على أي رجل أخرى'

In [66]:
dev_examples = dataset.dev_examples
len(dev_examples)

1224

In [46]:
dev_examples[10]

{
  "src": "يقولون بأنني أملك جمال كنز وطني .",
  "trg": "يقولون بأنني أملك جمال كنز وطني .",
  "src_g": "B",
  "trg_g": "B"
}

In [47]:
generate_sentence(mle, dev_examples[10].src, dev_examples[10].trg_g)

{'يقولون': 1.0}
{'بأنني': 1.0}
{'أملك': 1.0}
{'<unk>': 0.0}
{'<unk>': 0.0}
{'<unk>': 0.0}
{'.': 1.0}


'يقولون بأنني أملك <unk> <unk> <unk> .'

In [64]:
def output_preds(model, examples, output_dir):
    refs = []
    preds = []
    a = open(output_dir, 'w', encoding='utf8')
    for ex in examples:
        src = ex.src
        trg = ex.trg
        src_g = ex.src_g
        trg_g = ex.trg_g
        
        pred = generate_sentence(model, src, trg_g)
        a.write(pred)
        a.write('\n')

In [65]:
output_preds(mle, train_examples, \
'/home/ba63/gender-bias/christine_2019/Arabic-parallel-gender-corpus/edits_annotations/baseline.train.M.outputs')

In [67]:
output_preds(mle, dev_examples, \
'/home/ba63/gender-bias/christine_2019/Arabic-parallel-gender-corpus/edits_annotations/baseline.dev.M.outputs')

In [69]:
test_examples = dataset.test_examples
len(dev_examples)

1224

In [70]:
output_preds(mle, test_examples, \
'/home/ba63/gender-bias/christine_2019/Arabic-parallel-gender-corpus/edits_annotations/baseline.test.M.outputs')